Yuexin Yu<br>
Siliang Ma<br>
Yipeng Huang<br>
Jonathan Rodriguez-Perez

<h1> Group 3 - Dream Homes NYC - Checkpoint 4 </h1>

### Import relevant libraries

In [5]:
#pip install psycopg

In [7]:
import random
from datetime import date, time, timedelta

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import psycopg, os

### Connect to PostgreSQL DB

In [12]:
print('Connecting to the PostgreSQL database...')
conn = psycopg.connect(
    host="localhost",
    port='5432',
    dbname="Dream_Homes2", # make sure to create Dream_Homes2 database in pgAmin4 before connecting or running this cell
    user="postgres",
    password="123")

cur = conn.cursor()

print('PostgreSQL database version:')
cur.execute('SELECT version()')

db_version = cur.fetchone()
print(db_version)
cur.close()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 17.2 (Debian 17.2-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)


### Create tables based on Schema

In [15]:
cur = conn.cursor()

cur.execute('''

CREATE TABLE Offices (
    Office_ID INT PRIMARY KEY,
    address VARCHAR,
    city VARCHAR,
    state VARCHAR,
    zip_code VARCHAR,
    operating_hours VARCHAR,
    phone_number VARCHAR
);

CREATE TABLE Employees (
    Employee_ID INT PRIMARY KEY,
    Office_ID INT,
    first_name VARCHAR,
    last_name VARCHAR,
    employment_type VARCHAR,
    email VARCHAR,
    phone VARCHAR,
    FOREIGN KEY (Office_ID) REFERENCES Offices(Office_ID)
);

CREATE TABLE Agent_Schedules (
    Schedule_ID INT PRIMARY KEY,
    Employee_ID INT,
    day VARCHAR,
    start_time TIME,
    end_time TIME,
    FOREIGN KEY (Employee_ID) REFERENCES Employees(Employee_ID)
);

CREATE TABLE Compensation (
    Compensation_ID INT PRIMARY KEY,
    Employee_ID INT,
    salary DECIMAL,
    commission_rate DECIMAL,
    bonuses DECIMAL,
    effective_date DATE,
    FOREIGN KEY (Employee_ID) REFERENCES Employees(Employee_ID)
);

CREATE TABLE Clients (
    Client_ID INT PRIMARY KEY,
    first_name VARCHAR,
    last_name VARCHAR,
    client_type VARCHAR,
    email VARCHAR,
    phone VARCHAR
);

CREATE TABLE Client_Preferences (
    Preference_ID INT PRIMARY KEY,
    Client_ID INT,
    property_type VARCHAR,
    budget_range VARCHAR,
    preferred_location VARCHAR,
    additional_preferences TEXT,
    FOREIGN KEY (Client_ID) REFERENCES Clients(Client_ID)
);

CREATE TABLE Property_Location (
    Property_ID INT PRIMARY KEY,
    property_type VARCHAR,
    address VARCHAR,
    city VARCHAR,
    state VARCHAR,
    zip_code VARCHAR,
    price DECIMAL,
    status VARCHAR,
    listing_date DATE
);

CREATE TABLE Property_Features (
    Feature_ID INT PRIMARY KEY,
    Property_ID INT,
    bedrooms INT,
    bathrooms INT,
    square_footage INT,
    amenities TEXT,
    FOREIGN KEY (Property_ID) REFERENCES Property_Location(Property_ID)
);

CREATE TABLE Transactions (
    Transaction_ID INT PRIMARY KEY,
    Property_ID INT,
    Client_ID INT,
    Employee_ID INT,
    transaction_type VARCHAR,
    price DECIMAL,
    transaction_date DATE,
    closing_details TEXT,
    FOREIGN KEY (Property_ID) REFERENCES Property_Location(Property_ID),
    FOREIGN KEY (Client_ID) REFERENCES Clients(Client_ID),
    FOREIGN KEY (Employee_ID) REFERENCES Employees(Employee_ID)
);

CREATE TABLE Appointments (
    Appointment_ID INT PRIMARY KEY,
    date DATE,
    time TIME,
    Client_ID INT,
    Employee_ID INT,
    Property_ID INT,
    FOREIGN KEY (Client_ID) REFERENCES Clients(Client_ID),
    FOREIGN KEY (Employee_ID) REFERENCES Employees(Employee_ID),
    FOREIGN KEY (Property_ID) REFERENCES Property_Location(Property_ID)
);

CREATE TABLE Open_Houses (
    Open_House_ID INT PRIMARY KEY,
    Property_ID INT,
    scheduled_date DATE,
    time TIME,
    Employee_ID INT,
    visitor_count INT,
    feedback TEXT,
    FOREIGN KEY (Property_ID) REFERENCES Property_Location(Property_ID),
    FOREIGN KEY (Employee_ID) REFERENCES Employees(Employee_ID)
);

CREATE TABLE Marketing_Campaigns (
    Campaign_ID INT PRIMARY KEY,
    Office_ID INT,
    campaign_name VARCHAR,
    type VARCHAR,
    start_date DATE,
    end_date DATE,
    cost DECIMAL,
    leads_generated INT,
    FOREIGN KEY (Office_ID) REFERENCES Offices(Office_ID)
);

CREATE TABLE Expenses (
    Expense_ID INT PRIMARY KEY,
    Office_ID INT,
    expense_type VARCHAR,
    amount DECIMAL,
    date DATE,
    description TEXT,
    FOREIGN KEY (Office_ID) REFERENCES Offices(Office_ID)
);

CREATE TABLE Revenue (
    Revenue_ID INT PRIMARY KEY,
    Office_ID INT,
    amount DECIMAL,
    date DATE,
    description TEXT,
    FOREIGN KEY (Office_ID) REFERENCES Offices(Office_ID)
);

CREATE TABLE Schools (
    School_ID INT PRIMARY KEY,
    name VARCHAR,
    district VARCHAR,
    rating DECIMAL,
    city VARCHAR,
    state VARCHAR
);

CREATE TABLE Property_Schools (
    Property_ID INT,
    School_ID INT,
    PRIMARY KEY (Property_ID, School_ID),
    FOREIGN KEY (Property_ID) REFERENCES Property_Location(Property_ID),
    FOREIGN KEY (School_ID) REFERENCES Schools(School_ID)
);

''')

# Commit and close
conn.commit()
cur.close()

### Code to insert dummy data into all tables

In [18]:
# The current insert statements are being upgraded to include:
# - Realistic addresses and zip codes
# - 15 offices (5 each in NY, NJ, CT)
# - 150 employees (10 per office)
# - Full dependency-aware insert order (Offices → Employees → Clients, etc.)

cur = conn.cursor()

# Define max employee ID from employee insert loop
emp_id = 1
NUM_EMPLOYEES = 150  # constant to lock in employee count

# START: Expanded Inserts (Batch 1) — Offices, Employees, Agent_Schedules

# Offices
cur.execute("INSERT INTO Offices VALUES (1, '11 Broadway', 'Manhattan', 'NY', '10004', '9AM-5PM', '212-555-0101')")
cur.execute("INSERT INTO Offices VALUES (2, '55 Water St', 'Brooklyn', 'NY', '11201', '9AM-5PM', '718-555-0102')")
cur.execute("INSERT INTO Offices VALUES (3, '31-00 47th Ave', 'Queens', 'NY', '11101', '9AM-5PM', '718-555-0103')")
cur.execute("INSERT INTO Offices VALUES (4, '800 River Ave', 'Bronx', 'NY', '10451', '9AM-5PM', '718-555-0104')")
cur.execute("INSERT INTO Offices VALUES (5, '100 Wall St', 'Manhattan', 'NY', '10005', '9AM-5PM', '212-555-0105')")
cur.execute("INSERT INTO Offices VALUES (6, '1 Newark Center', 'Newark', 'NJ', '07102', '9AM-5PM', '973-555-0106')")
cur.execute("INSERT INTO Offices VALUES (7, '2 Exchange Pl', 'Jersey City', 'NJ', '07302', '9AM-5PM', '201-555-0107')")
cur.execute("INSERT INTO Offices VALUES (8, '221 River St', 'Hoboken', 'NJ', '07030', '9AM-5PM', '201-555-0108')")
cur.execute("INSERT INTO Offices VALUES (9, '100 Bayard St', 'New Brunswick', 'NJ', '08901', '9AM-5PM', '732-555-0109')")
cur.execute("INSERT INTO Offices VALUES (10, '500 Broad St', 'Elizabeth', 'NJ', '07201', '9AM-5PM', '908-555-0110')")
cur.execute("INSERT INTO Offices VALUES (11, '1 Landmark Sq', 'Stamford', 'CT', '06901', '9AM-5PM', '203-555-0111')")
cur.execute("INSERT INTO Offices VALUES (12, '185 Asylum St', 'Hartford', 'CT', '06103', '9AM-5PM', '860-555-0112')")
cur.execute("INSERT INTO Offices VALUES (13, '157 Church St', 'New Haven', 'CT', '06510', '9AM-5PM', '203-555-0113')")
cur.execute("INSERT INTO Offices VALUES (14, '90 State House Sq', 'Hartford', 'CT', '06123', '9AM-5PM', '860-555-0114')")
cur.execute("INSERT INTO Offices VALUES (15, '20 Danbury Rd', 'Wilton', 'CT', '06897', '9AM-5PM', '203-555-0115')")


# Employees (10 per office, 150 total)
for office_id in range(1, 16):
    for i in range(10):
        emp_type = 'Full-Time' if random.random() < 0.7 else 'Part-Time'
        cur.execute(f"INSERT INTO Employees VALUES ({emp_id}, {office_id}, 'First{emp_id}', 'Last{emp_id}', '{emp_type}', 'employee{emp_id}@dh.com', '555-01{emp_id:03}')")
        emp_id += 1

cur.execute("SELECT COUNT(*) FROM Employees")
print("Employees inserted:", cur.fetchone()[0])
conn.commit()

# Compensation (1 per employee)
comp_id = 1
for e_id in range(1, NUM_EMPLOYEES + 1):
    salary = round(random.uniform(55000, 150000), 2)
    commission = round(random.uniform(0.01, 0.10), 2)
    bonus = round(random.uniform(1000, 10000), 2)
    eff_date = f"2024-{random.randint(1,12):02}-{random.randint(1,28):02}"
    cur.execute(f"INSERT INTO Compensation VALUES ({comp_id}, {e_id}, {salary}, {commission}, {bonus}, '{eff_date}')")
    comp_id += 1

conn.commit()

# Agent_Schedules (1 per employee)
sched_id = 1
for agent_id in range(1, NUM_EMPLOYEES + 1):
    cur.execute(f"INSERT INTO Agent_Schedules VALUES ({sched_id}, {agent_id}, '2025-04-01', '09:00', '17:00')")
    sched_id += 1

conn.commit()


# Clients (10 clients per agent_id)
client_id = 1
for agent_id in range(1, NUM_EMPLOYEES + 1):  # 1 to 150 agents
    for i in range(10):  # 10 clients per agent
        client_type = 'Buyer' if random.random() < 0.5 else 'Seller'
        cur.execute(f"""
            INSERT INTO Clients 
            (client_id, first_name, last_name, client_type, email, phone)
            VALUES 
            ({client_id}, 'First{client_id}', 'Last{client_id}', '{client_type}', 'client{client_id}@dh.com', '555-02{client_id:03}')
        """)
        client_id += 1

conn.commit()


# Client_Preferences (1 per client)
pref_id = 1
for cid in range(1, 1501):
    property_types = ['House', 'Condo', 'Apartment', 'Townhouse']
    locations = ['Manhattan', 'Brooklyn', 'Hoboken', 'Hartford', 'Stamford', 'Queens']
    budget = ['$300k-$500k', '$500k-$700k', '$700k-$900k']
    extra = ['Near subway', 'Pet-friendly', 'Low HOA', 'Backyard', 'Parking included']

    import random
    cur.execute(f"INSERT INTO Client_Preferences VALUES ({pref_id}, {cid}, '{random.choice(property_types)}', '{random.choice(budget)}', '{random.choice(locations)}', '{random.choice(extra)}')")
    pref_id += 1

# Property_Location (120 total: 40 NY, 40 NJ, 40 CT)
property_id = 1
addresses = [
    # NY
    ('House', '345 Park Ave', 'Manhatan', 'NY', '10154', 1200000, 'Available', '2025-04-01'),
    ('Condo', '172 Classon Ave', 'Brooklyn', 'NY', '11205', 800000, 'Sold', '2025-04-02'),
    ('Apartment', '41-42 24th St', 'Queens', 'NY', '11101', 900000, 'Available', '2025-04-03'),
    ('Townhouse', '780 Grand Concourse', 'Bronx', 'NY', '10451', 2950000, 'Pending', '2025-04-04'),
    ('House', '10 Liberty St', 'Manhattan', 'NY', '10005', 3500000, 'Available', '2025-04-05'),
    # NJ
    ('Condo', '765 Broad St', 'Newark', 'NJ', '07102', 720000, 'Available', '2025-04-06'),
    ('Apartment', '88 Morgan St', 'Jersey City', 'NJ', '07302', 650000, 'Sold', '2025-04-07'),
    ('Townhouse', '1025 Washington St', 'Hoboken', 'NJ', '07030', 370000, 'Available', '2025-04-08'),
    ('House', '100 Somerset St', 'New Brunswick', 'NJ', '08901', 780000, 'Pending', '2025-04-09'),
    ('Condo', '600 Elizabeth Ave', 'Elizabeth', 'NJ', '07201', 210000, 'Available', '2025-04-10'),
    # CT
    ('House', '1 Harbor Point Rd', 'Stamford', 'CT', '06902', 1100000, 'Available', '2025-04-11'),
    ('Townhouse', '555 Main St', 'Hartford', 'CT', '06103', 850000, 'Sold', '2025-04-12'),
    ('Apartment', '900 Chapel St', 'New Haven', 'CT', '06510', 590000, 'Pending', '2025-04-13'),
    ('Condo', '45 Pratt St', 'Hartford', 'CT', '06123', 720000, 'Available', '2025-04-14'),
    ('House', '25 Old Ridgefield Rd', 'Wilton', 'CT', '06897', 930000, 'Available', '2025-04-15')
]

# Expand with slight variation to make 40 per state
for state_offset in range(3):  # NY, NJ, CT
    base = state_offset * 40
    for i in range(40):
        t = addresses[state_offset * 5 + (i % 5)]
        cur.execute(f"""
            INSERT INTO Property_Location VALUES (
                {property_id}, '{t[0]}', '{t[1]} #{i+1}', '{t[2]}', '{t[3]}', '{t[4]}', {t[5]}, '{t[6]}', '{t[7]}'
            )
        """)
        property_id += 1

conn.commit()

# Property_Features (4–5 features per property)
import random
features_pool = [
    'Garage', 'Balcony', 'Fireplace', 'Hardwood Floors', 'Renovated Kitchen',
    'Backyard', 'Finished Basement', 'Smart Home', 'Solar Panels', 'Walk-in Closet',
    'Open Floor Plan', 'Central Air', 'Stainless Steel Appliances', 'Pool', 'Deck'
]

feature_id = 1
for pid in range(1, 121):  # Property_IDs from 1 to 120
    selected = random.sample(features_pool, k=random.randint(4, 5))
    for feat in selected:
        beds = random.randint(1, 5)
        baths = random.randint(1, 3)
        sqft = random.randint(800, 3000)
        cur.execute(f"INSERT INTO Property_Features VALUES ({feature_id}, {pid}, {beds}, {baths}, {sqft}, '{feat}')")
        feature_id += 1
conn.commit()


import datetime
from random import randint
import random

# Transactions (Sale vs Rent with correct FK handling)
transaction_id = 1
inserted_transactions = 0

valid_property_ids = list(range(1, 121))  # from Property_Location
valid_client_ids = list(range(1, NUM_EMPLOYEES * 10 + 1))  # from Clients
valid_employee_ids = list(range(1, NUM_EMPLOYEES + 1))  # from Employees

while inserted_transactions < 150:  # 150 transactions
    property_id = random.choice(valid_property_ids)
    client_id = random.choice(valid_client_ids)
    employee_id = random.choice(valid_employee_ids)
    
    transaction_type = 'Sale' if random.random() < 0.6 else 'Rent'

    if transaction_type == 'Sale':
        price = round(random.uniform(300000, 2000000), 2)
        closing_details = random.choice([
            'Luxury Sale', 'Quick Close', 'Above Asking Price', 'Cash Buyer', 'Multiple Offers'
        ])
    else:
        price = round(random.uniform(1500, 10000), 2)
        rented_days = random.randint(5, 60)
        closing_details = f"Rented in {rented_days} days"

    transaction_date = f"2024-{randint(1,12):02}-{randint(1,28):02}"

    try:
        cur.execute(f"""
            INSERT INTO Transactions (Transaction_ID, Property_ID, Client_ID, Employee_ID, transaction_type, price, transaction_date, closing_details)
            VALUES ({transaction_id}, {property_id}, {client_id}, {employee_id}, '{transaction_type}', {price}, '{transaction_date}', '{closing_details}')
        """)
        transaction_id += 1
        inserted_transactions += 1
    except Exception:
        conn.rollback()

conn.commit()



#appointments
import datetime
from random import randint
import random

from psycopg.errors import ForeignKeyViolation

# Fetch real IDs from database
cur.execute("SELECT client_id FROM Clients")
client_ids = [row[0] for row in cur.fetchall()]

cur.execute("SELECT property_id FROM Property_Location")
property_ids = [row[0] for row in cur.fetchall()]

appointment_id = 1
successful = 0

while successful < 50:
    date = f"2025-{randint(1,12):02}-{randint(1,28):02}"
    time = f"{randint(9,16):02}:00"
    agent_id = randint(1, NUM_EMPLOYEES)
    client_id = random.choice(client_ids)
    property_id = random.choice(property_ids)

    try:
        cur.execute(f"""
            INSERT INTO Appointments VALUES
            ({appointment_id}, '{date}', '{time}', {agent_id}, {client_id}, {property_id})
        """)
        conn.commit()
        #print(f"[INSERTED] appointment_id={appointment_id} with agent={agent_id}, client={client_id}, property={property_id}")
        appointment_id += 1
        successful += 1
    except ForeignKeyViolation:
        conn.rollback()
        continue

    except Exception as e:
        conn.rollback()
        print(f"[ERROR] {e}")

cur.execute("SELECT MAX(employee_id), COUNT(*) FROM Employees")
print("[CHECK] Max employee_id, Total employees:", cur.fetchone())




# Open_Houses (one per property)
open_house_id = 1
for pid in range(1, 121):
    scheduled_date = f"2025-04-{random.randint(1, 28):02}"
    time = f"{random.randint(9, 17):02}:00"
    employee_id = random.randint(1, NUM_EMPLOYEES)
    visitors = random.randint(3, 25)
    feedback = random.choice(['Very interested', 'Needs more space', 'Too expensive', 'Perfect location', 'Not enough light'])
    cur.execute(f"INSERT INTO Open_Houses VALUES ({open_house_id}, {pid}, '{scheduled_date}', '{time}', {employee_id}, {visitors}, '{feedback}')")
    open_house_id += 1

# Marketing_Campaigns (multiple per office)
campaign_id = 1
channels = ['Social Media', 'Email', 'TV', 'Billboards', 'Flyers', 'Google Ads']
types = ['Online', 'Offline']
for office_id in range(1, 16):
    for _ in range(2):
        campaign = f"{random.choice(channels)} Campaign"
        channel_type = random.choice(types)
        budget = random.randint(500, 3000)
        start_date = f"2025-03-{random.randint(1, 15):02}"
        end_date = f"2025-04-{random.randint(10, 30):02}"
        cur.execute(f"INSERT INTO Marketing_Campaigns VALUES ({campaign_id}, {office_id}, '{campaign}', '{channel_type}', '{start_date}', '{end_date}', {budget}, {budget//10})")
        campaign_id += 1

# Expenses (multiple per office)
expense_id = 1
categories = ['Utilities', 'Internet', 'Supplies', 'Maintenance', 'Software']
for office_id in range(1, 16):
    for _ in range(3):
        category = random.choice(categories)
        amount = random.randint(100, 2000)
        expense_date = f"2025-04-{random.randint(1, 28):02}"
        cur.execute(f"INSERT INTO Expenses VALUES ({expense_id}, {office_id}, '{category}', {amount}, '{expense_date}')")
        expense_id += 1

# Schools — sample inserts for new schema
school_id = 1
school_types = ['Public', 'Private', 'Charter']
cities = ['Manhattan', 'Brooklyn', 'Queens', 'Stamford', 'Hartford']

for i in range(10):
    name = f"School {i+1}"
    district = f"District {random.randint(1, 5)}"
    rating = round(random.uniform(2.5, 5.0), 1)
    city = random.choice(cities)
    state = 'NY' if city in ['Manhattan', 'Brooklyn', 'Queens'] else 'CT'
    cur.execute(f"INSERT INTO Schools VALUES ({school_id}, '{name}', '{district}', {rating}, '{city}', '{state}')")
    school_id += 1

conn.commit()

# Property_Schools — avoid duplicates
linked_pairs = set()
link_attempts = 0

while len(linked_pairs) < 30 and link_attempts < 100:
    pid = random.randint(1, 120)
    sid = random.randint(1, 10)
    if (pid, sid) not in linked_pairs:
        try:
            cur.execute(f"INSERT INTO Property_Schools VALUES ({pid}, {sid})")
            linked_pairs.add((pid, sid))
        except psycopg.errors.UniqueViolation:
            conn.rollback()
    link_attempts += 1

conn.commit()


# Revenue (multiple entries per office)
revenue_id = 1
for office_id in range(1, 16):
    for _ in range(2):
        amount = random.randint(50000, 200000)
        date = f"2025-04-{random.randint(1, 28):02}"
        desc = random.choice(['Q1 Sales', 'Commission Revenue', 'Rental Income', 'Bonus Payout'])
        cur.execute(f"INSERT INTO Revenue VALUES ({revenue_id}, {office_id}, {amount}, '{date}', '{desc}')")
        revenue_id += 1



conn.commit()
cur.close()
print("Manual inserts completed.")


Employees inserted: 150
[CHECK] Max employee_id, Total employees: (150, 150)
Manual inserts completed.


# View tables and dummy data; before continuing to the SQL Query portion of project:

In [21]:
cur = conn.cursor()

# Fetch all tables
cur.execute("""
    SELECT table_name FROM information_schema.tables
    WHERE table_schema = 'public'
""")
tables = cur.fetchall()

for table in tables:
    print(f"\nTable: {table[0]}")
    cur.execute(f"SELECT * FROM {table[0]} LIMIT 5")
    rows = cur.fetchall()
    for row in rows:
        print(row)

cur.close()
#conn.close()


Table: offices
(1, '11 Broadway', 'Manhattan', 'NY', '10004', '9AM-5PM', '212-555-0101')
(2, '55 Water St', 'Brooklyn', 'NY', '11201', '9AM-5PM', '718-555-0102')
(3, '31-00 47th Ave', 'Queens', 'NY', '11101', '9AM-5PM', '718-555-0103')
(4, '800 River Ave', 'Bronx', 'NY', '10451', '9AM-5PM', '718-555-0104')
(5, '100 Wall St', 'Manhattan', 'NY', '10005', '9AM-5PM', '212-555-0105')

Table: employees
(1, 1, 'First1', 'Last1', 'Full-Time', 'employee1@dh.com', '555-01001')
(2, 1, 'First2', 'Last2', 'Part-Time', 'employee2@dh.com', '555-01002')
(3, 1, 'First3', 'Last3', 'Full-Time', 'employee3@dh.com', '555-01003')
(4, 1, 'First4', 'Last4', 'Full-Time', 'employee4@dh.com', '555-01004')
(5, 1, 'First5', 'Last5', 'Full-Time', 'employee5@dh.com', '555-01005')

Table: agent_schedules
(1, 1, '2025-04-01', datetime.time(9, 0), datetime.time(17, 0))
(2, 2, '2025-04-01', datetime.time(9, 0), datetime.time(17, 0))
(3, 3, '2025-04-01', datetime.time(9, 0), datetime.time(17, 0))
(4, 4, '2025-04-01', da

In [23]:
# 1. Office Operations Overview
cur = conn.cursor()
cur.execute("""
SELECT o.office_id, o.address, o.city, o.state, o.operating_hours,
       COUNT(DISTINCT e.employee_id) AS num_employees,
       COUNT(DISTINCT s.schedule_id) AS num_schedules
FROM Offices o
LEFT JOIN Employees e ON o.office_id = e.office_id
LEFT JOIN Agent_Schedules s ON e.employee_id = s.employee_id
GROUP BY o.office_id;
""")

columns1 = [desc[0] for desc in cur.description]
data1 = cur.fetchall()
df1 = pd.DataFrame(data1, columns=columns1)

# Display the DataFrame
df1.head()


,office_id,address,city,state,operating_hours,num_employees,num_schedules
0,1,11 Broadway,Manhattan,NY,9AM-5PM,10,10
1,2,55 Water St,Brooklyn,NY,9AM-5PM,10,10
2,3,31-00 47th Ave,Queens,NY,9AM-5PM,10,10
3,4,800 River Ave,Bronx,NY,9AM-5PM,10,10
4,5,100 Wall St,Manhattan,NY,9AM-5PM,10,10


In [25]:
# 2. Agent Compensation Summary
cur = conn.cursor()
cur.execute("""
SELECT e.employee_id, e.first_name, e.last_name, e.employment_type,
       c.salary, c.commission_rate, c.bonuses, c.effective_date
FROM Employees e
LEFT JOIN Compensation c ON e.employee_id = c.employee_id;
""")

columns2 = [desc[0] for desc in cur.description]
data2 = cur.fetchall()
df2 = pd.DataFrame(data2, columns=columns2)

# Display the DataFrame
df2.head()


,employee_id,first_name,last_name,employment_type,salary,commission_rate,bonuses,effective_date
0,1,First1,Last1,Full-Time,86963.6,0.03,2421.8,2024-07-01
1,2,First2,Last2,Part-Time,73515.52,0.05,9079.23,2024-02-18
2,3,First3,Last3,Full-Time,110580.94,0.06,4751.79,2024-04-02
3,4,First4,Last4,Full-Time,60788.22,0.06,5924.93,2024-10-22
4,5,First5,Last5,Full-Time,98857.54,0.09,9385.76,2024-12-23


In [27]:
# 3. Client Preferences
cur = conn.cursor()
cur.execute("""
SELECT c.client_id, c.first_name, c.last_name, c.client_type,
       cp.property_type, cp.budget_range, cp.preferred_location
FROM Clients c
LEFT JOIN Client_Preferences cp ON c.client_id = cp.client_id;
""")

columns3 = [desc[0] for desc in cur.description]
data3 = cur.fetchall()
df3 = pd.DataFrame(data3, columns=columns3)

# Display the DataFrame
df3.head()

,client_id,first_name,last_name,client_type,property_type,budget_range,preferred_location
0,1,First1,Last1,Buyer,Townhouse,$500k-$700k,Brooklyn
1,2,First2,Last2,Buyer,House,$300k-$500k,Hoboken
2,3,First3,Last3,Buyer,House,$300k-$500k,Hartford
3,4,First4,Last4,Buyer,Condo,$300k-$500k,Stamford
4,5,First5,Last5,Seller,House,$300k-$500k,Hoboken


In [29]:
# 4. Active Property Listings
cur = conn.cursor()
cur.execute("""
SELECT property_id, address, city, state, price, status, listing_date
FROM Property_Location
WHERE status = 'Available';
""")

columns4 = [desc[0] for desc in cur.description]
data4 = cur.fetchall()
df4 = pd.DataFrame(data4, columns=columns4)

# Display the DataFrame
df4.head()


,property_id,address,city,state,price,status,listing_date
0,1,345 Park Ave #1,Manhatan,NY,1200000,Available,2025-04-01
1,3,41-42 24th St #3,Queens,NY,900000,Available,2025-04-03
2,5,10 Liberty St #5,Manhattan,NY,3500000,Available,2025-04-05
3,6,345 Park Ave #6,Manhatan,NY,1200000,Available,2025-04-01
4,8,41-42 24th St #8,Queens,NY,900000,Available,2025-04-03


In [31]:
# 5. Transaction Overview
cur = conn.cursor()
cur.execute("""
SELECT t.transaction_id, p.address,
       c.first_name || ' ' || c.last_name AS client,
       e.first_name || ' ' || e.last_name AS agent,
       t.transaction_type, t.price, t.transaction_date
FROM Transactions t
JOIN Property_Location p ON t.property_id = p.property_id
JOIN Clients c ON t.client_id = c.client_id
JOIN Employees e ON t.employee_id = e.employee_id;
""")

columns5 = [desc[0] for desc in cur.description]
data5 = cur.fetchall()
df5 = pd.DataFrame(data5, columns=columns5)

# Display the DataFrame
df5.head()


,transaction_id,address,client,agent,transaction_type,price,transaction_date
0,84,555 Main St #32,First5 Last5,First103 Last103,Rent,3246.01,2024-01-18
1,52,100 Somerset St #39,First16 Last16,First13 Last13,Sale,639694.55,2024-09-24
2,48,172 Classon Ave #7,First33 Last33,First62 Last62,Sale,710642.7,2024-06-15
3,124,345 Park Ave #1,First46 Last46,First121 Last121,Sale,1001087.91,2024-05-04
4,75,765 Broad St #6,First46 Last46,First86 Last86,Sale,737137.99,2024-06-22


In [33]:
# 6. Upcoming Appointments (Fixed test date: 2025-04-04)
cur = conn.cursor()
cur.execute("""
SELECT a.appointment_id, a.date, a.time,
       c.first_name AS client, p.address AS property
FROM Appointments a
JOIN Clients c ON a.client_id = c.client_id
JOIN Property_Location p ON a.property_id = p.property_id
WHERE a.date >= '2025-04-04'
ORDER BY a.date, a.time;
""")

columns6 = [desc[0] for desc in cur.description]
data6 = cur.fetchall()
df6 = pd.DataFrame(data6, columns=columns6)

# Display the DataFrame
df6.head()


,appointment_id,date,time,client,property
0,18,2025-04-08,10:00:00,First141,1025 Washington St #33
1,29,2025-04-11,12:00:00,First27,10 Liberty St #5
2,28,2025-04-12,15:00:00,First72,1 Harbor Point Rd #1
3,37,2025-04-13,11:00:00,First132,900 Chapel St #8
4,26,2025-04-20,12:00:00,First70,600 Elizabeth Ave #10


In [35]:
# 7. Upcoming Open Houses (Fixed test date: 2025-04-04)
cur = conn.cursor()
cur.execute("""
SELECT oh.open_house_id, p.address, oh.scheduled_date, oh.time, oh.visitor_count
FROM Open_Houses oh
JOIN Property_Location p ON oh.property_id = p.property_id
WHERE oh.scheduled_date >= '2025-04-04'
ORDER BY oh.scheduled_date, oh.time;
""")

columns7 = [desc[0] for desc in cur.description]
data7 = cur.fetchall()
df7 = pd.DataFrame(data7, columns=columns7)

# Display the DataFrame
df7.head()


,open_house_id,address,scheduled_date,time,visitor_count
0,107,555 Main St #27,2025-04-04,17:00:00,3
1,93,900 Chapel St #13,2025-04-05,10:00:00,16
2,47,88 Morgan St #7,2025-04-06,13:00:00,23
3,37,172 Classon Ave #37,2025-04-07,09:00:00,3
4,17,172 Classon Ave #17,2025-04-07,11:00:00,6


In [37]:
# 8. Recommended Properties by Preference (based on cp.budget_range column)
cur = conn.cursor()
cur.execute("""
SELECT cp.client_id, c.first_name, c.last_name,
       p.property_id, p.address, p.city, p.price
FROM Client_Preferences cp
JOIN Clients c ON cp.client_id = c.client_id
JOIN Property_Location p
  ON cp.property_type = p.property_type
 AND cp.preferred_location = p.city
WHERE p.status = 'Available'
  AND p.price <= (
    REPLACE(REPLACE(SPLIT_PART(cp.budget_range, '-', 2), '$', ''), 'k', '')::NUMERIC * 1000
);
""")

columns8 = [desc[0] for desc in cur.description]
data8 = cur.fetchall()
df8 = pd.DataFrame(data8, columns=columns8)

# Display the DataFrame
df8.head()




,client_id,first_name,last_name,property_id,address,city,price
0,67,First67,Last67,119,45 Pratt St #39,Hartford,720000
1,67,First67,Last67,114,45 Pratt St #34,Hartford,720000
2,67,First67,Last67,109,45 Pratt St #29,Hartford,720000
3,67,First67,Last67,104,45 Pratt St #24,Hartford,720000
4,67,First67,Last67,99,45 Pratt St #19,Hartford,720000


In [39]:
# 9. Top Performing Agents
cur = conn.cursor()
cur.execute("""
SELECT e.employee_id, e.first_name, e.last_name,
       COUNT(t.transaction_id) AS total_transactions,
       SUM(t.price) AS total_sales
FROM Employees e
JOIN Transactions t ON e.employee_id = t.employee_id
GROUP BY e.employee_id
ORDER BY total_sales DESC;
""")

columns9 = [desc[0] for desc in cur.description]
data9 = cur.fetchall()
df9 = pd.DataFrame(data9, columns=columns9)

# Display the DataFrame
df9.head()


,employee_id,first_name,last_name,total_transactions,total_sales
0,54,First54,Last54,3,5130058.38
1,86,First86,Last86,5,4430493.19
2,124,First124,Last124,3,3591277.09
3,132,First132,Last132,3,3564682.31
4,117,First117,Last117,4,3515437.30


In [41]:
# 10. Office Profitability
cur = conn.cursor()
cur.execute("""
SELECT o.office_id, o.city,
       COALESCE(SUM(r.amount), 0) AS total_revenue,
       COALESCE(SUM(e.amount), 0) AS total_expenses,
       COALESCE(SUM(r.amount), 0) - COALESCE(SUM(e.amount), 0) AS net_profit
FROM Offices o
LEFT JOIN Revenue r ON o.office_id = r.office_id
LEFT JOIN Expenses e ON o.office_id = e.office_id
GROUP BY o.office_id
ORDER BY net_profit DESC;
""")

columns10 = [desc[0] for desc in cur.description]
data10 = cur.fetchall()
df10 = pd.DataFrame(data10, columns=columns10)

# Display the DataFrame
df10.head()


,office_id,city,total_revenue,total_expenses,net_profit
0,13,New Haven,832968,6878,826090
1,9,New Brunswick,822735,8502,814233
2,3,Queens,805668,2476,803192
3,11,Stamford,794025,3230,790795
4,12,Hartford,793440,6002,787438


In [43]:
# 11. Marketing Campaign Effectiveness
cur = conn.cursor()
cur.execute("""
SELECT campaign_name, type, start_date, end_date, cost, leads_generated,
       CASE WHEN cost > 0 THEN leads_generated::DECIMAL / cost ELSE NULL END AS leads_per_dollar
FROM Marketing_Campaigns;
""")

columns11 = [desc[0] for desc in cur.description]
data11 = cur.fetchall()
df11 = pd.DataFrame(data11, columns=columns11)

# Display the DataFrame
df11.head()


,campaign_name,type,start_date,end_date,cost,leads_generated,leads_per_dollar
0,TV Campaign,Online,2025-03-05,2025-04-12,1476,147,0.09959349593495934959
1,Flyers Campaign,Offline,2025-03-14,2025-04-19,2589,258,0.09965237543453070684
2,Google Ads Campaign,Offline,2025-03-15,2025-04-29,592,59,0.09966216216216216216
3,Google Ads Campaign,Online,2025-03-15,2025-04-19,956,95,0.09937238493723849372
4,TV Campaign,Online,2025-03-04,2025-04-12,1626,162,0.09963099630996309963


In [45]:
# 12. Market Trends (Derived from existing tables)
cur = conn.cursor()
cur.execute("""
SELECT city,
       ROUND(AVG(CURRENT_DATE - listing_date)) AS average_listing_duration_days,
       ROUND(AVG(price)) AS average_listing_price,
       COUNT(*) FILTER (WHERE status = 'Available') AS available_count,
       COUNT(*) FILTER (WHERE status = 'Sold') AS sold_count,
       COUNT(*) FILTER (WHERE status = 'Rented') AS rented_count,
       CURRENT_DATE AS recorded_date
FROM Property_Location
WHERE listing_date <= CURRENT_DATE
GROUP BY city
HAVING COUNT(*) > 0
ORDER BY average_listing_price DESC;
""")

columns12 = [desc[0] for desc in cur.description]
data12 = cur.fetchall()
df12 = pd.DataFrame(data12, columns=columns12)

# Display the DataFrame
df12.head()


,city,average_listing_duration_days,average_listing_price,available_count,sold_count,rented_count,recorded_date
0,Manhattan,22,3500000,8,0,0,2025-04-27
1,Bronx,23,2950000,0,0,0,2025-04-27
2,Manhatan,26,1200000,8,0,0,2025-04-27
3,Stamford,16,1100000,8,0,0,2025-04-27
4,Wilton,12,930000,8,0,0,2025-04-27


In [47]:
cur.close()

In [49]:
conn.close()